# Fine-tuning a pretrained model

## [Introduction](https://huggingface.co/course/chapter3/1?fw=pt)
In [Chapter 2](https://huggingface.co/course/chapter2) we explored how to use tokenizers and pretrained models to make predictions. But what if you want to fine-tune a pretrained model for your own dataset? That's the topic of this chapter! You will learn:
- How to prepare a large dataset from the Hub
- How to use the high-level `Trainer` API to fine-tune a model
- How to use a custom training loop
- How to leverage the 🤗 Accelerate library to easily run that custom training loop on any distributed setup

In order to upload your trained checkpoints to the Hugging Face Hub, you will need a huggingface.co account: [create an account](https://huggingface.co/join)!

## [Processing the data](https://huggingface.co/course/chapter3/2?fw=pt)
Continuing with the example from the [previous chapter](https://huggingface.co/course/chapter2), here is how we would train a sequence classifier on one batch in PyTorch:

In [1]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification
# Same as before
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!"
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
# This is new
batch["labels"] = torch.tensor([1, 1]) # 1 = positive
optimizer = AdamW(model.parameters())  # optimizer
loss = model(**batch).loss             # get loss
loss.backward()                        # backpropagation
optimizer.step()                       # optimization step (no logits => warning below)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Of course, just training the model on two sentences is not going to yield very good results. To get better results, you will need to prepare a bigger dataset.

In this section, we will use as an example the MRPC (Microsoft Research Paraphrase Corpus) dataset, introduced in a [paper](https://www.aclweb.org/anthology/I05-5002.pdf) by William B. Dolan and Chris Brockett. The dataset consists of 5,801 pairs of sentences, with a label indicating if they are paraphrases or not (i.e., if both sentences mean the same thing). We've selected it for this chapter because it's a small dataset, so it's easy to experiment with training on it.

### Loading a dataset from the Hub

In [2]:
from IPython.display import HTML
HTML('<iframe width="640" height="360" src="https://www.youtube.com/embed/_BZearw7f0w" allowfullscreen></iframe>')

/home/matthias/anaconda3/envs/hf/lib/python3.9/site-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


The Hub doesn't just contain models; it also has multiple datasets in lots of different languages. You can browse the datasets [here](https://huggingface.co/datasets), and we recommend you try to load and process a new dataset once you have gone through this section (see the general documentation [here](https://huggingface.co/docs/datasets/loading_datasets.html#from-the-huggingface-hub)). But for now, let's focus on the MRPC dataset! This is one of the 10 datasets composing the [GLUE benchmark](https://gluebenchmark.com/), which is an academic benchmark that is used to measure the performance of ML models across 10 different text classification tasks.

The 🤗 Datasets library provides a very simple command to download and cache a dataset on the Hub. We can download the MRPC dataset like this:

In [3]:
from datasets import load_dataset
raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

Reusing dataset glue (/home/matthias/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

As you can see, we get a `DatasetDict` object which contains the training set, the validation set, and the test set. Each of those contains several columns (`sentence1`, `sentence2`, `label`, and `idx`) and a variable number of rows, which are the number of elements in each set (so, there are 3,668 pairs of sentences in the training set, 408 in the validation set, and 1,725 in the test set).

This command downloads and caches the dataset, by default in *~/.cache/huggingface/dataset*. Recall from Chapter 2 that you can customize your cache folder by setting the `HF_HOME` environment variable.

We can access each pair of sentences in our `raw_datasets` object by indexing, like with a dictionary:

In [4]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

We can see the labels are already integers, so we won't have to do any preprocessing there. To know which integer corresponds to which label, we can inspect the `features` of our `raw_train_dataset`. This will tell us the type of each column:

In [5]:
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

Behind the scenes, label is of type ClassLabel, and the mapping of integers to label name is stored in the names folder. 0 corresponds to not_equivalent, and 1 corresponds to equivalent.
> ✏️ Try it out! <font color="darkgreen">Look at element 15 of the training set and element 87 of the validation set. What are their labels?</font>

In [6]:
# Trying it out
raw_valid_dataset = raw_datasets["validation"]
raw_train_dataset[15], raw_valid_dataset[87] # both labels are 0 ("not_equivalent")

({'sentence1': 'Rudder was most recently senior vice president for the Developer & Platform Evangelism Business .',
  'sentence2': 'Senior Vice President Eric Rudder , formerly head of the Developer and Platform Evangelism unit , will lead the new entity .',
  'label': 0,
  'idx': 16},
 {'sentence1': 'However , EPA officials would not confirm the 20 percent figure .',
  'sentence2': 'Only in the past few weeks have officials settled on the 20 percent figure .',
  'label': 0,
  'idx': 812})

### Preprocessing a dataset

In [7]:
HTML('<iframe width="640" height="360" src="https://www.youtube.com/embed/0u3ioSwev3s" allowfullscreen></iframe>')

/home/matthias/anaconda3/envs/hf/lib/python3.9/site-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


To preprocess the dataset, we need to convert the text to numbers the model can make sense of. As you saw in the [previous chapter](https://huggingface.co/course/chapter2), this is done with a tokenizer. We can feed the tokenizer one sentence or a list of sentences, so we can directly tokenize all the first sentences and all the second sentences of each pair like this:

In [8]:
#from transformers import AutoTokenizer
#checkpoint = "bert-base-uncased"
#tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_sentences_1 = tokenizer(raw_datasets["train"]["sentence1"])
tokenized_sentences_2 = tokenizer(raw_datasets["train"]["sentence2"])

However, we can't just pass two sequences to the model and get a prediction of whether the two sentences are paraphrases or not. We need to handle the two sequences as a pair, and apply the appropriate preprocessing. Fortunately, the tokenizer can also take a pair of sequences and prepare it the way our BERT model expects:

In [9]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

We discussed the `input_ids` and `attention_mask` keys in [Chapter 2](https://huggingface.co/course/chapter2), but we put off talking about `token_type_ids`. In this example, this is what tells the model which part of the input is the first sentence and which is the second sentence.
> ✏️ Try it out! <font color="darkgreen">Take element 15 of the training set and tokenize the two sentences separately and as a pair. What’s the difference between the two results?</font>

In [10]:
# Trying it out
try_dataset_train = raw_datasets["train"]
try_instance_15_train = try_dataset_train[15]
# sentence 1
try_instance_15_train_sent_1 = try_instance_15_train["sentence1"]
sent_1_tokenized = tokenizer(try_instance_15_train_sent_1)
print("sentence 1 tokenized:\n{}\n".format(sent_1_tokenized))
# sentence 2
try_instance_15_train_sent_2 = try_instance_15_train["sentence2"]
sent_2_tokenized = tokenizer(try_instance_15_train_sent_2)
print("sentence 2 tokenized:\n{}\n".format(sent_2_tokenized))
# sentences 1 & 2
sents_1_and_2_tokenized = tokenizer(try_instance_15_train_sent_1, try_instance_15_train_sent_2) # No list!
print("sentences 1 and 2 tokenized:\n{}".format(sents_1_and_2_tokenized))

sentence 1 tokenized:
{'input_ids': [101, 24049, 2001, 2087, 3728, 3026, 3580, 2343, 2005, 1996, 9722, 1004, 4132, 9340, 12439, 2964, 2449, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

sentence 2 tokenized:
{'input_ids': [101, 3026, 3580, 2343, 4388, 24049, 1010, 3839, 2132, 1997, 1996, 9722, 1998, 4132, 9340, 12439, 2964, 3131, 1010, 2097, 2599, 1996, 2047, 9178, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

sentences 1 and 2 tokenized:
{'input_ids': [101, 24049, 2001, 2087, 3728, 3026, 3580, 2343, 2005, 1996, 9722, 1004, 4132, 9340, 12439, 2964, 2449, 1012, 102, 3026, 3580, 2343, 4388, 24049, 1010, 3839, 2132, 1997, 1996, 9722, 1998, 4132, 9340, 12439, 2964, 3131, 1010, 2097, 2599, 1996, 2047, 9178, 1012

If we decode the IDs inside `input_ids` back to words, we will get:

In [11]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[CLS]',
 'this',
 'is',
 'the',
 'first',
 'sentence',
 '.',
 '[SEP]',
 'this',
 'is',
 'the',
 'second',
 'one',
 '.',
 '[SEP]']

So we see the model expects the inputs to be of the form `[CLS] sentence1 [SEP] sentence2 [SEP]` when there are two sentences. Aligning this with the `token_type_ids` gives us:

In [12]:
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"])
token_ids = inputs["token_type_ids"]
for i in range(len(tokens)):
    print("{}\tid: {}".format(tokens[i], token_ids[i]))

[CLS]	id: 0
this	id: 0
is	id: 0
the	id: 0
first	id: 0
sentence	id: 0
.	id: 0
[SEP]	id: 0
this	id: 1
is	id: 1
the	id: 1
second	id: 1
one	id: 1
.	id: 1
[SEP]	id: 1


As you can see, the parts of the input corresponding to `[CLS] sentence1 [SEP]` all have a token type ID of `0`, while the other parts, corresponding to `sentence2 [SEP]`, all have a token type ID of `1`.

Note that if you select a different checkpoint, you won't necessarily have the `token_type_ids` in your tokenized inputs (for instance, they're not returned if you use a DistilBERT model). They are only returned when the model will know what to do with them, because it has seen them during its pretraining.

Here, BERT is pretrained with token type IDs, and on top of the masked language modeling objective we talked about in [Chapter 1](https://huggingface.co/course/chapter1), it has an additional objective called *next sentence prediction*. The goal with this task is to model the relationship between pairs of sentences.

With next sentence prediction, the model is provided pairs of sentences (with randomly masked tokens) and asked to predict whether the second sentence follows the first. To make the task non-trivial, half of the time the sentences follow each other in the original document they were extracted from, and the other half of the time the two sentences come from two different documents.

In general, you don't need to worry about whether or not there are `token_type_ids` in your tokenized inputs: as long as you use the same checkpoint for the tokenizer and the model, everything will be fine as the tokenizer knows what to provide to its model.

Now that we have seen how our tokenizer can deal with one pair of sentences, we can use it to tokenize our whole dataset: like in the [previous chapter](https://huggingface.co/course/chapter2), we can feed the tokenizer a list of pairs of sentences by giving it the list of first sentences, then the list of second sentences. This is also compatible with the padding and truncation options we saw in [Chapter 2](https://huggingface.co/course/chapter2). So, one way to preprocess the training dataset is:

In [13]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)

This works well, but it has the disadvantage of returning a dictionary (with our `keys`, `input_ids`, `attention_mask`, and `token_type_ids`, and values that are lists of lists). It will also only work if you have enough RAM to store your whole dataset during the tokenization (whereas the datasets from the 🤗 Datasets library are [Apache Arrow](https://arrow.apache.org/) files stored on the disk, so you only keep the samples you ask for loaded in memory).

To keep the data as a dataset, we will use the [`Dataset.map()`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map) method. This also allows us some extra flexibility, if we need more preprocessing done than just tokenization. The `map()` method works by applying a function on each element of the dataset, so let's define a function that tokenizes our inputs:

In [14]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

This function takes a dictionary (like the items of our dataset) and returns a new dictionary with the keys `input_ids`, `attention_mask`, and `token_type_ids`. Note that it also works if the `example` dictionary contains several samples (each key as a list of sentences) since the `tokenizer` works on lists of pairs of sentences, as seen before. This will allow us to use the option `batched=True` in our call to `map()`, which will greatly speed up the tokenization. The `tokenizer` is backed by a tokenizer written in Rust from the [🤗 Tokenizers](https://github.com/huggingface/tokenizers) library. This tokenizer can be very fast, but only if we give it lots of inputs at once.

Note that we've left the `padding` argument out in our tokenization function for now. This is because padding all the samples to the maximum length is not efficient: it's better to pad the samples when we're building a batch, as then we only need to pad to the maximum length in that batch, and not the maximum length in the entire dataset. This can save a lot of time and processing power when the inputs have very variable lengths!

Here is how we apply the tokenization function on all our datasets at once. We're using `batched=True` in our call to `map` so the function is applied to multiple elements of our dataset at once, and not on each element separately. This allows for faster preprocessing.

The way the 🤗 Datasets library applies this processing is by adding new fields to the datasets, one for each key in the dictionary returned by the preprocessing function:

In [15]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Loading cached processed dataset at /home/matthias/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-c4a19653f5f97871.arrow
Loading cached processed dataset at /home/matthias/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-48163fec37dd1675.arrow
Loading cached processed dataset at /home/matthias/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-fa73f7cbf643098b.arrow


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

You can even use multiprocessing when applying your preprocessing function with `map()` by passing along a `num_proc` argument. We didn't do this here because the 🤗 Tokenizers library already uses multiple threads to tokenize our samples faster, but if you are not using a fast tokenizer backed by this library, this could speed up your preprocessing.

Our `tokenize_function` returns a dictionary with the keys `input_ids`, `attention_mask`, and `token_type_ids`, so those three fields are added to all splits of our dataset. Note that we could also have changed existing fields if our preprocessing function returned a new value for an existing key in the dataset to which we applied `map()`.

The last thing we will need to do is pad all the examples to the length of the longest element when we batch elements together — a technique we refer to as *dynamic padding*.

### Dynamic padding

In [16]:
HTML('<iframe width="640" height="360" src="https://www.youtube.com/embed/7q5NyFT8REg" allowfullscreen></iframe>')

The function that is responsible for putting together samples inside a batch is called a *collate function*. It's an argument you can pass when you build a `DataLoader`, the default being a function that will just convert your samples to PyTorch tensors and concatenate them (recursively if your elements are lists, tuples, or dictionaries). This won't be possible in our case since the inputs we have won't all be of the same size. We have deliberately postponed the padding, to only apply it as necessary on each batch and avoid having over-long inputs with a lot of padding. This will speed up training by quite a bit, but note that if you're training on a TPU it can cause problems — TPUs prefer fixed shapes, even when that requires extra padding.

To do this in practice, we have to define a collate function that will apply the correct amount of padding to the items of the dataset we want to batch together. Fortunately, the 🤗 Transformers library provides us with such a function via `DataCollatorWithPadding`. It takes a tokenizer when you instantiate it (to know which padding token to use, and whether the model expects padding to be on the left or on the right of the inputs) and will do everything you need:

In [17]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_collator

DataCollatorWithPadding(tokenizer=PreTrainedTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}), padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

To test this new toy, let's grab a few samples from our training set that we would like to batch together. Here, we remove the columns `idx`, `sentence1`, and `sentence2` as they won't be needed and contain strings (and we can't create tensors with strings) and have a look at the lengths of each entry in the batch:

In [18]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]

[50, 59, 47, 67, 59, 50, 62, 32]

No surprise, we get samples of varying length, from 32 to 67. Dynamic padding means the samples in this batch should all be padded to a length of 67, the maximum length inside the batch. Without dynamic padding, all of the samples would have to be padded to the maximum length in the whole dataset, or the maximum length the model can accept. Let's double-check that our `data_collator` is dynamically padding the batch properly:

In [19]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 67]),
 'token_type_ids': torch.Size([8, 67]),
 'attention_mask': torch.Size([8, 67]),
 'labels': torch.Size([8])}

Looking good! Now that we've gone from raw text to batches our model can deal with, we're ready to fine-tune it!
> ✏️ Try it out! <font color="darkgreen">Replicate the preprocessing on the GLUE SST-2 dataset. It's a little bit different since it's composed of single sentences instead of pairs, but the rest of what we did should look the same. For a harder challenge, try to write a preprocessing function that works on any of the GLUE tasks.</font>

In [20]:
# Trying it out
## GLUE SST-2 (https://huggingface.co/datasets/glue/viewer/sst2)
### download and explore dataset
sst2_raw_datasets = load_dataset("glue", "sst2")          # get the entire dataset
sst2_train = sst2_raw_datasets["train"]                   # train split (for exploration)
print("SST-2 features:\n{}".format(sst2_train.features))  # features
print("\nsentence 0:\n{}".format(sst2_train[0]))          # example sentence
### tokenization as demonstrated above
def sst2_tokenize_function(item):                         # tokenization function for .map method
    return tokenizer(item["sentence"], truncation=True)
print("\nsentence 0, tokenized:\n{}".format(sst2_tokenize_function(sst2_train[0])))   # tokenized example sentence
tokenized_sst2_datasets = sst2_raw_datasets.map(sst2_tokenize_function, batched=True) # batch-tokenize all datasets
print("\nbatch-tokenized SST-2:\n{}".format(tokenized_sst2_datasets))
### datacollator and batching
sst2_train_samples = tokenized_sst2_datasets["train"][:3] # get first 3 tokenized samples of the training set
print("\nfirst 3 samples of the training set:\n{}".format(sst2_train_samples))        # check keys in output below
sst2_train_purged = {k: v for k, v in sst2_train_samples.items() if k not in ["idx", "sentence"]}
print("\nfirst 3 samples of the training set without unnecessary keys:\n{}".format(sst2_train_purged))
print("\nthe first 3 samples of the training set have different lengths:\n{}".format(
    [len(x) for x in sst2_train_purged["input_ids"]]
))
sst2_train_batch = data_collator(sst2_train_purged)       # use data_collator to turn samples into a batch
print("\ndatacollator turns samples of different length into a batch with constant lengths:\n{}".format(
    {k: v.shape for k, v in sst2_train_batch.items()}
))
### generic code for all GLUE tasks is shown in the next code cell

Reusing dataset glue (/home/matthias/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/matthias/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-674853c1b801ccb3.arrow
Loading cached processed dataset at /home/matthias/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-d35a4986800bf902.arrow
Loading cached processed dataset at /home/matthias/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-658479390570bfd0.arrow


SST-2 features:
{'sentence': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=2, names=['negative', 'positive'], id=None), 'idx': Value(dtype='int32', id=None)}

sentence 0:
{'sentence': 'hide new secretions from the parental units ', 'label': 0, 'idx': 0}

sentence 0, tokenized:
{'input_ids': [101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

batch-tokenized SST-2:
DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1821
    })
})

first 3 samples of the training set:
{'

> ✏️ Still trying it out: <font color="darkgreen">Generic code for batching any of the glue tasks (as suggested above) is shown below.</font>

In [21]:
# Still trying it out (generic code for all GLUE tasks)
def glue_tokenize_function(fields):
    if len(fields)==2:
        return lambda instance: tokenizer(instance[fields[0]], instance[fields[1]], truncation=True)
    return lambda instance: tokenizer(instance[fields[0]], truncation=True)
def prepare_glue_datasets(name):
    raw_data = load_dataset("glue", name)
    subsets = list(raw_data.keys())
    print("available splits:\t{}".format(subsets))
    keys = list(raw_data[subsets[0]].features.keys())
    tokenized_raw_data = raw_data.map(glue_tokenize_function(keys[:-2]), batched=True)
    return tokenized_raw_data
def build_glue_dataset_batch(name, shard, batch_size=8):
    print("glue subdataset:\t{}\nsubdataset split:\t{}".format(name, shard))
    tokenized_glue_datasets = prepare_glue_datasets(name)
    tokenized_glue_dataset = tokenized_glue_datasets[shard]
    tokenized_glue_dataset_batch = tokenized_glue_dataset[:batch_size]
    glue_keep_list = ["label", "input_ids", "token_type_ids", "attention_mask"]
    return {k: v for k, v in tokenized_glue_dataset_batch.items() if k in glue_keep_list}
# possible glue dataset options: ...
# ... ["cola", "sst2", "mrpc", "qqp", "stsb", "mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "ax"]
my_uneven_glue_batch = build_glue_dataset_batch("mnli", "train", batch_size=8)
print([len(x) for x in my_uneven_glue_batch["input_ids"]]) # mrpc (train): [50, 59, 47, 67, 59, 50, 62, 32]
my_glue_batch = data_collator(my_uneven_glue_batch)        # use data_collator to turn samples into a batch
{k: v.shape for k, v in my_glue_batch.items()}             # mrpc (train): 8x67

glue subdataset:	mnli
subdataset split:	train


Reusing dataset glue (/home/matthias/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/5 [00:00<?, ?it/s]

Loading cached processed dataset at /home/matthias/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-a2a8695418a0469a.arrow
Loading cached processed dataset at /home/matthias/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-79148a6795d7b43b.arrow
Loading cached processed dataset at /home/matthias/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-3eba5101c3305251.arrow
Loading cached processed dataset at /home/matthias/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-7ee63a46c8b79e91.arrow
Loading cached processed dataset at /home/matthias/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-678cbb9a49489c5f.arrow


available splits:	['train', 'validation_matched', 'validation_mismatched', 'test_matched', 'test_mismatched']
[28, 78, 28, 21, 44, 44, 61, 27]


{'input_ids': torch.Size([8, 78]),
 'token_type_ids': torch.Size([8, 78]),
 'attention_mask': torch.Size([8, 78]),
 'labels': torch.Size([8])}

## [Fine-tuning a model with the Trainer API](https://huggingface.co/course/chapter3/3?fw=pt)

In [22]:
HTML('<iframe width="640" height="360" src="https://www.youtube.com/embed/nvBXf7s7vTI" allowfullscreen></iframe>')

/home/matthias/anaconda3/envs/hf/lib/python3.9/site-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


🤗 Transformers provides a `Trainer` class to help you fine-tune any of the pretrained models it provides on your dataset. Once you've done all the data preprocessing work in the last section, you have just a few steps left to define the `Trainer`. The hardest part is likely to be preparing the environment to run `Trainer.train()`, as it will run very slowly on a CPU. If you don't have a GPU set up, you can get access to free GPUs or TPUs on Google Colab.

The code examples below assume you have already executed the examples in the previous section. Here is a short summary recapping what you need:

In [23]:
raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Reusing dataset glue (/home/matthias/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/matthias/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-80396165d446312f.arrow
Loading cached processed dataset at /home/matthias/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-01380b7bd47c61ee.arrow
Loading cached processed dataset at /home/matthias/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-1439f53624f4cb1c.arrow


### Training
The first step before we can define our `Trainer` is to define a `TrainingArguments` class that will contain all the hyperparameters the `Trainer` will use for training and evaluation. The only argument you have to provide is a directory where the trained model will be saved, as well as the checkpoints along the way. For all the rest, you can leave the defaults, which should work pretty well for a basic fine-tuning.

In [24]:
from transformers import TrainingArguments
training_args = TrainingArguments("test-trainer")

> <font color="darkgreen">💡 If you want to automatically upload your model to the Hub during training, pass along `push_to_hub=True` in the `TrainingArguments`. We will learn more about this in [Chapter 4](https://huggingface.co/course/chapter4/3).</font>

The second step is to define our model. As in the [previous chapter](https://huggingface.co/course/chapter2), we will use the `AutoModelForSequenceClassification` class, with two labels:

In [25]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

You will notice that unlike in [Chapter 2](https://huggingface.co/course/chapter2), you get a warning after instantiating this pretrained model. This is because BERT has not been pretrained on classifying pairs of sentences, so the head of the pretrained model has been discarded and a new head suitable for sequence classification has been added instead. The warnings indicate that some weights were not used (the ones corresponding to the dropped pretraining head) and that some others were randomly initialized (the ones for the new head). It concludes by encouraging you to train the model, which is exactly what we are going to do now.

Once we have our model, we can define a `Trainer` by passing it all the objects constructed up to now — the `model`, the `training_args`, the training and validation datasets, our `data_collator`, and our `tokenizer`:

In [26]:
from transformers import Trainer
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer
)
trainer

Note that when you pass the `tokenizer` as we did here, the default `data_collator` used by the `Trainer` will be a `DataCollatorWithPadding` as defined previously, so you can skip the line `data_collator=data_collator` in this call. It was still important to show you this part of the processing in section 2!

To fine-tune the model on our dataset, we just have to call the `train()` method of our `Trainer`:

In [27]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, idx, sentence2.
***** Running training *****
  Num examples = 3668
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1377


Step,Training Loss
500,0.516100
1000,0.306900


Saving model checkpoint to test-trainer/checkpoint-500
Configuration saved in test-trainer/checkpoint-500/config.json
Model weights saved in test-trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-1000
Configuration saved in test-trainer/checkpoint-1000/config.json
Model weights saved in test-trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-1000/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1377, training_loss=0.3396895747299028, metrics={'train_runtime': 139.6326, 'train_samples_per_second': 78.807, 'train_steps_per_second': 9.862, 'total_flos': 405470580750720.0, 'train_loss': 0.3396895747299028, 'epoch': 3.0})

In [28]:
# RTX 2060 + GTX 1050ti:        6min, 44sec
# RTX 2060             :        2min, 36sec
# RTX 3060             :        2min,  5sec
# GTX 1050ti           :        7min, 32sec
# i5-7600K             :       30min, 53sec
# M1                   : 2hrs, 51min, 59sec

This will start the fine-tuning (which should take a couple of minutes on a GPU) and report the training loss every 500 steps. It won't, however, tell you how well (or badly) your model is performing. This is because:

1. We didn't tell the `Trainer` to evaluate during training by setting `evaluation_strategy` to either `"steps"` (evaluate every `eval_steps`) or `"epoch"` (evaluate at the end of each epoch).
1. We didn’t provide the `Trainer` with a `compute_metrics()` function to calculate a metric during said evaluation (otherwise the evaluation would just have printed the loss, which is not a very intuitive number).

### Evaluation
Let's see how we can build a useful `compute_metrics()` function and use it the next time we train. The function must take an `EvalPrediction` object (which is a named tuple with a `predictions` field and a `label_ids` field) and will return a dictionary mapping strings to floats (the strings being the names of the metrics returned, and the floats their values). To get some predictions from our model, we can use the `Trainer.predict()` command:

In [29]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, idx, sentence2.
***** Running Prediction *****
  Num examples = 408
  Batch size = 8


(408, 2) (408,)


The output of the `predict()` method is another named tuple with three fields: `predictions`, `label_ids`, and `metrics`. The `metrics` field will just contain the loss on the dataset passed, as well as some time metrics (how long it took to predict, in total and on average). Once we complete our `compute_metrics()` function and pass it to the `Trainer`, that field will also contain the metrics returned by `compute_metrics()`.

As you can see, `predictions` is a two-dimensional array with shape 408 x 2 (408 being the number of elements in the dataset we used). Those are the logits for each element of the dataset we passed to `predict()` (as you saw in the [previous chapter](https://huggingface.co/course/chapter2), all Transformer models return logits). To transform them into predictions that we can compare to our labels, we need to take the index with the maximum value on the second axis:

In [30]:
import numpy as np
preds = np.argmax(predictions.predictions, axis=-1)

We can now compare those `preds` to the labels. To build our `compute_metric()` function, we will rely on the metrics from the 🤗 Datasets library. We can load the metrics associated with the MRPC dataset as easily as we loaded the dataset, this time with the `load_metric()` function. The object returned has a `compute()` method we can use to do the metric calculation:

In [31]:
from datasets import load_metric
metric = load_metric("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8529411764705882, 'f1': 0.8979591836734693}

The exact results you get may vary, as the random initialization of the model head might change the metrics it achieved. Here, we can see our model has an accuracy of 85.78% on the validation set and an $F_1$ score of 89.97%. Those are the two metrics used to evaluate results on the MRPC dataset for the GLUE benchmark. The table in the [BERT paper](https://arxiv.org/pdf/1810.04805.pdf) reported an $F_1$ score of 88.9% for the base model. That was the uncased model while we are currently using the cased model, which explains the better result.

Wrapping everything together, we get our `compute_metrics()` function:

In [32]:
def compute_metrics(eval_preds):
    metric = load_metric("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

And to see it used in action to report metrics at the end of each epoch, here is how we define a new `Trainer` with this `compute_metrics()` function:

In [33]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/matthias/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads"

Note that we create a new `TrainingArguments` with its `evaluation_strategy` set to `"epoch"` and a new model — otherwise, we would just be continuing the training of the model we have already trained. To launch a new training run, we execute:

In [34]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, idx, sentence2.
***** Running training *****
  Num examples = 3668
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1377


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.551957,0.693627,0.816984
2,0.625500,0.468524,0.838235,0.886598
3,0.471600,0.546619,0.843137,0.891156


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, idx, sentence2.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 8
Saving model checkpoint to test-trainer/checkpoint-500
Configuration saved in test-trainer/checkpoint-500/config.json
Model weights saved in test-trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, idx, sentence2.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 8
Saving model checkpoint to test-trainer/checkpoint-1000
Configuration saved in test-trainer/checkpoint-1000/config.json
Model weights saved in test-trainer/checkpoin

TrainOutput(global_step=1377, training_loss=0.48934927658491056, metrics={'train_runtime': 146.1922, 'train_samples_per_second': 75.271, 'train_steps_per_second': 9.419, 'total_flos': 405470580750720.0, 'train_loss': 0.48934927658491056, 'epoch': 3.0})

In [35]:
# RTX 2060 + GTX 1050ti:        7min,  6sec
# RTX 2060             :        2min, 48sec
# RTX 3060             :        2min, 13sec
# GTX 1050ti           : cuda oom
# i5-7600K             :       31min, 36sec
# M1                   : 2hrs, 58min, 38sec

This time, it will report the validation loss and metrics at the end of each epoch on top of the training loss. Again, the exact accuracy/$F_1$ score you reach might be a bit different from what we found, because of the random head initialization of the model, but it should be in the same ballpark.

The `Trainer` will work out of the box on multiple GPUs or TPUs and provides lots of options, like mixed-precision training (use `fp16 = True` in your training arguments). We will go over everything it supports in Chapter 10.

This concludes the introduction to fine-tuning using the `Trainer` API. An example of doing this for most common NLP tasks will be given in Chapter 7, but for now let's look at how to do the same thing in pure PyTorch.
> ✏️ Try it out! <font color="darkgreen">Fine-tune a model on the GLUE SST-2 dataset, using the data processing you did in section 2.</font>

In [36]:
# Trying it out
## checkpoint
checkpoint = "bert-base-uncased"
## data preprocessing (to batches => tokenizer, datacollator, etc.)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
sst2_raw_datasets = load_dataset("glue", "sst2")
def sst2_tokenize_function(item):
    return tokenizer(item["sentence"], truncation=True)
tokenized_sst2_datasets = sst2_raw_datasets.map(sst2_tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
## training qrguments
training_args = TrainingArguments(
    # https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments
    "trainer-try_1",
    evaluation_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16
)
## model (first try the same, then a new one)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)
## compute_metrics
def compute_metrics(eval_preds):
    metric = load_metric("glue", "sst2")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
## trainer
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
## train
trainer.train()

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/matthias/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /hom

  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/matthias/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-c2cad20fdea9ff2c.arrow
Loading cached processed dataset at /home/matthias/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-0885ddbcac993665.arrow
Loading cached processed dataset at /home/matthias/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-faa394fa930547ac.arrow
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/matthias/.cache/huggingf

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.379163,0.848039
2,No log,0.407240,0.833333
3,0.404500,0.585537,0.840686
4,0.404500,0.736674,0.848039
5,0.069700,0.769874,0.850490


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, idx, sentence2.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, idx, sentence2.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 16
Saving model checkpoint to trainer-try_1/checkpoint-500
Configuration saved in trainer-try_1/checkpoint-500/config.json
Model weights saved in trainer-try_1/checkpoint-500/pytorch_model.bin
tokenizer config file saved in trainer-try_1/checkpoint-500/tokenizer_config.json
Special tokens file saved in trainer-try_1/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence1, 

TrainOutput(global_step=1150, training_loss=0.20919458285621975, metrics={'train_runtime': 176.2127, 'train_samples_per_second': 104.079, 'train_steps_per_second': 6.526, 'total_flos': 716681893993128.0, 'train_loss': 0.20919458285621975, 'epoch': 5.0})

## [A full training](https://huggingface.co/course/chapter3/4?fw=pt)

In [37]:
HTML('<iframe width="640" height="360" src="https://www.youtube.com/embed/Dh9CL8fyG80" allowfullscreen></iframe>')

/home/matthias/anaconda3/envs/hf/lib/python3.9/site-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


Now we'll see how to achieve the same results as we did in the last section without using the `Trainer` class. Again, we assume you have done the data processing in section 2. Here is a short summary covering everything you will need:

In [38]:
raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Reusing dataset glue (/home/matthias/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/matthias/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /hom

### Prepare for training
Before actually writing our training loop, we will need to define a few objects. The first ones are the dataloaders we will use to iterate over batches. But before we can define those dataloaders, we need to apply a bit of postprocessing to our `tokenized_datasets`, to take care of some things that the `Trainer` did for us automatically. Specifically, we need to:
- Remove the columns corresponding to values the model does not expect (like the `sentence1` and `sentence2` columns).
- Rename the column `label` to `labels` (because the model expects the argument to be named `labels`).
- Set the format of the datasets so they return PyTorch tensors instead of lists.

Our `tokenized_datasets` has one method for each of those steps. We can then check that the result only has columns that our model will accept:

In [39]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

Now that this is done, we can easily define our dataloaders:

In [40]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator)
eval_dataloader = DataLoader(tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator)

To quickly check there is no mistake in the data processing, we can inspect a batch like this:

In [41]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 73]),
 'token_type_ids': torch.Size([8, 73]),
 'attention_mask': torch.Size([8, 73])}

Note that the actual shapes will probably be slightly different for you since we set `shuffle=True` for the training dataloader and we are padding to the maximum length inside the batch.

Now that we're completely finished with data preprocessing (a satisfying yet elusive goal for any ML practitioner), let's turn to the model. We instantiate it exactly as we did in the previous section:

In [42]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/matthias/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin fr

To make sure that everything will go smoothly during training, we pass our batch to this model:

In [43]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.7616, grad_fn=<NllLossBackward>) torch.Size([8, 2])


All 🤗 Transformers models will return the loss when `labels` are provided, and we also get the logits (two for each input in our batch, so a tensor of size $8\times2$).

We're almost ready to write our training loop! We're just missing two things: an optimizer and a learning rate scheduler. Since we are trying to replicate what the `Trainer` was doing by hand, we will use the same defaults. The optimizer used by the `Trainer` is `AdamW`, which is the same as `Adam`, but with a twist for weight decay regularization (see [“Decoupled Weight Decay Regularization”](https://arxiv.org/abs/1711.05101) by Ilya Loshchilov and Frank Hutter):

In [44]:
optimizer = AdamW(model.parameters(), lr=5e-5)

Finally, the learning rate scheduler used by default is just a linear decay from the maximum value (5e-5) to 0. To properly define it, we need to know the number of training steps we will take, which is the number of epochs we want to run multiplied by the number of training batches (which is the length of our training dataloader). The `Trainer` uses three epochs by default, so we will follow that:

In [45]:
from transformers import get_scheduler
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
num_training_steps

1377

### The training loop
One last thing: we will want to use the GPU if we have access to one (on a CPU, training might take several hours instead of a couple of minutes). To do this, we define a device we will put our model and our batches on:

In [46]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

We are now ready to train! To get some sense of when training will be finished, we add a progress bar over our number of training steps, using the `tqdm` library:

In [47]:
from tqdm.auto import tqdm
progress_bar = tqdm(range(num_training_steps))
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/1377 [00:00<?, ?it/s]

In [48]:
# RTX 2060 + GTX 1050ti: cuda oom
# RTX 2060             : cuda oom
# RTX 3060             :        2min,  2sec
# GTX 1050ti           : cuda oom
# i5-7600K             :       29min, 23sec
# M1                   : 2hrs, 51min, 38sec

You can see that the core of the training loop looks a lot like the one in the introduction. We didn't ask for any reporting, so this training loop will not tell us anything about how the model fares. We need to add an evaluation loop for that.

### The evaluation loop
As we did earlier, we will use a metric provided by the 🤗 Datasets library. We've already seen the `metric.compute()` method, but metrics can actually accumulate batches for us as we go over the prediction loop with the method `add_batch()`. Once we have accumulated all the batches, we can get the final result with `metric.compute()`. Here's how to implement all of this in an evaluation loop:

In [49]:
metric = load_metric("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])
metric.compute()

{'accuracy': 0.8529411764705882, 'f1': 0.8979591836734693}

Again, your results will be slightly different because of the randomness in the model head initialization and the data shuffling, but they should be in the same ballpark.
> ✏️ Try it out! <font color="darkgreen">Modify the previous training loop to fine-tune your model on the SST-2 dataset.</font>

In [50]:
# Trying it out
## load dataset, tokenize, and apply datacollator
sst2_raw_datasets = load_dataset("glue", "sst2")          # get the entire dataset
sst2_checkpoint = "bert-base-uncased"
sst2_tokenizer = AutoTokenizer.from_pretrained(sst2_checkpoint)
def sst2_tokenize_function(item):                         # tokenization function for .map method
    return tokenizer(item["sentence"], truncation=True)
sst2_tokenized_datasets = sst2_raw_datasets.map(sst2_tokenize_function, batched=True)
sst2_data_collator = DataCollatorWithPadding(tokenizer=sst2_tokenizer)
## prepare tokenized datasets for training
sst2_tokenized_datasets = sst2_tokenized_datasets.remove_columns(["sentence", "idx"])
sst2_tokenized_datasets = sst2_tokenized_datasets.rename_column("label", "labels")
sst2_tokenized_datasets.set_format("torch")
print("column names:\n{}".format(sst2_tokenized_datasets["train"].column_names))
## build dataloaders
batch_size = 16
sst2_train_dataloader = DataLoader(
    sst2_tokenized_datasets["train"], shuffle=True, batch_size=batch_size, collate_fn=sst2_data_collator
)
sst2_eval_dataloader = DataLoader(
    sst2_tokenized_datasets["validation"], batch_size=batch_size, collate_fn=sst2_data_collator
)
## get single batch
for batch in sst2_train_dataloader:
    break
print("\nshapes within a batch:\n{}".format({k: v.shape for k, v in batch.items()}))
## define a model and run a batch through it
sst2_model = AutoModelForSequenceClassification.from_pretrained(sst2_checkpoint, num_labels=3)
sst2_outputs = sst2_model(**batch)
print("\nloss and logits shape:\nloss: {}\tlogits shape: {}".format(sst2_outputs.loss, sst2_outputs.logits.shape))
## optimizer and lr_scheduler
sst2_optimizer = AdamW(sst2_model.parameters(), lr=5e-5)
sst2_num_epochs = 3
sst2_num_training_steps = sst2_num_epochs * len(sst2_train_dataloader)
sst2_lr_scheduler = get_scheduler(
    "linear",
    optimizer=sst2_optimizer,
    num_warmup_steps=0,
    num_training_steps=sst2_num_training_steps,
)
print("\ntraining steps:\n{}".format(sst2_num_training_steps))
## set device
sst2_device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
sst2_model.to(sst2_device)
print("\nrun model on the following device:\n{}".format(sst2_device))
## model.train() loop
progress_bar = tqdm(range(sst2_num_training_steps))
sst2_model.train()
for epoch in range(sst2_num_epochs):
    for batch in sst2_train_dataloader:
        batch = {k: v.to(sst2_device) for k, v in batch.items()}
        outputs = sst2_model(**batch)
        loss = outputs.loss
        loss.backward()
        sst2_optimizer.step()
        sst2_lr_scheduler.step()
        sst2_optimizer.zero_grad()
        progress_bar.update(1)
## model.eval() loop
sst2_metric = load_metric("glue", "mrpc")
sst2_model.eval()
for batch in sst2_eval_dataloader:
    batch = {k: v.to(sst2_device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = sst2_model(**batch)
    sst2_logits = outputs.logits
    sst2_predictions = torch.argmax(sst2_logits, dim=-1)
    sst2_metric.add_batch(predictions=sst2_predictions, references=batch["labels"])
sst2_metric.compute()

Reusing dataset glue (/home/matthias/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/matthias/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /hom

column names:
['labels', 'input_ids', 'token_type_ids', 'attention_mask']

shapes within a batch:
{'labels': torch.Size([16]), 'input_ids': torch.Size([16, 34]), 'token_type_ids': torch.Size([16, 34]), 'attention_mask': torch.Size([16, 34])}


loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/matthias/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.0",
  "type_voc


loss and logits shape:
loss: 1.1249617338180542	logits shape: torch.Size([16, 3])

training steps:
12630

run model on the following device:
cuda


  0%|          | 0/12630 [00:00<?, ?it/s]

{'accuracy': 0.9231651376146789, 'f1': 0.9244644870349492}

### Supercharge your training loop with 🤗 Accelerate

In [51]:
HTML('<iframe width="640" height="360" src="https://www.youtube.com/embed/s7dy8QRgjJ0" allowfullscreen></iframe>')

The training loop we defined earlier works fine on a single CPU or GPU. But using the [🤗 Accelerate](https://github.com/huggingface/accelerate) library, with just a few adjustments we can enable distributed training on multiple GPUs or TPUs. Starting from the creation of the training and validation dataloaders, here is what our manual training loop looks like:
```python
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
progress_bar = tqdm(range(num_training_steps))
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
```
And here are the changes:
```python
from accelerate import Accelerator
accelerator = Accelerator() # +
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)
# - device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# - model.to(device)
train_dataloader, eval_dataloader, model, optimizer = accelerator.prepare( # +
    train_dataloader, eval_dataloader, model, optimizer                    # +
)                                                                          # +
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)
progress_bar = tqdm(range(num_training_steps))
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
# -     batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
# -     loss.backward()
        accelerator.backward(loss) # +
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
```
The first line to add is the import line. The second line instantiates an `Accelerator` object that will look at the environment and initialize the proper distributed setup. 🤗 Accelerate handles the device placement for you, so you can remove the lines that put the model on the device (or, if you prefer, change them to use `accelerator.device` instead of `device`).

Then the main bulk of the work is done in the line that sends the dataloaders, the model, and the optimizer to `accelerator.prepare()`. This will wrap those objects in the proper container to make sure your distributed training works as intended. The remaining changes to make are removing the line that puts the batch on the `device` (again, if you want to keep this you can just change it to use `accelerator.device`) and replacing `loss.backward()` with `accelerator.backward(loss)`.
> <font color="darkgreen">⚠️ In order to benefit from the speed-up offered by Cloud TPUs, we recommend padding your samples to a fixed length with the `padding="max_length"` and `max_length` arguments of the tokenizer.</font>

If you'd like to copy and paste it to play around, here's what the complete training loop looks like with 🤗 Accelerate:

In [52]:
from accelerate import Accelerator
accelerator = Accelerator()
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)
train_dl, eval_dl, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer
)
num_epochs = 3
num_training_steps = num_epochs * len(train_dl)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
progress_bar = tqdm(range(num_training_steps))
model.train()
for epoch in range(num_epochs):
    for batch in train_dl:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/matthias/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin fr

  0%|          | 0/1377 [00:00<?, ?it/s]

In [53]:
# RTX 2060 + GTX 1050ti: cuda oom
# RTX 2060             : cuda oom
# RTX 3060             :       2min  2sec
# GTX 1050ti           : cuda oom
# i5-7600K             :      29min 12sec
# M1                   : 3hrs 14min 46sec

Putting this in a `train.py` script will make that script runnable on any kind of distributed setup. To try it out in your distributed setup, run the command:
```bash
accelerate config
```
which will prompt you to answer a few questions and dump your answers in a configuration file used by this command:
```bash
accelerate launch train.py
```
which will launch the distributed training.

If you want to try this in a Notebook (for instance, to test it with TPUs on Colab), just paste the code in a `training_function()` and run a last cell with:
```python
from accelerate import notebook_launcher
notebook_launcher(training_function)
```
You can find more examples in the [🤗 Accelerate repo](https://github.com/huggingface/accelerate/tree/main/examples).

## [Fine-tuning, Check!](https://huggingface.co/course/chapter3/5?fw=pt)
That was fun! In the first two chapters you learned about models and tokenizers, and now you know how to fine-tune them for your own data. To recap, in this chapter you:
- Learned about datasets in the [Hub](https://huggingface.co/datasets).
- Learned how to load and preprocess datasets, including using dynamic padding and collators.
- Implemented your own fine-tuning and evaluation of a model.
- Implemented a lower-level training loop.
- Used 🤗 Accelerate to easily adapt your training loop so it works for multiple GPUs or TPUs.

## [End-of-chapter quiz](https://huggingface.co/course/chapter3/6?fw=pt)
Test what you learned in this chapter!

**1. The `emotion` dataset contains Twitter messages labeled with emotions. Search for it in the Hub, and read the dataset card. Which of these is not one of its basic emotions?**<br>
⚪️ Joy<br>
⚪️ Love<br>
⚫️ Confusion
> **Correct!** Correct! Confusion is not one of the six basic emotions.

⚪️ Surprise

**2. Search for the `ar_sarcasm` dataset in the [Hub](https://huggingface.co/datasets). Which task does it support?**<br>
⚫️ Sentiment classification
> **Correct!** That's right! You can tell thanks to the tags.

⚪️ Machine translation<br>
⚪️ Named entity recognition<br>
⚪️ Question answering

**3. How does the BERT model expect a pair of sentences to be processed?**<br>
⚪️ Tokens_of_sentence_1 [SEP] Tokens_of_sentence_2<br>
⚪️ [CLS] Tokens_of_sentence_1 Tokens_of_sentence_2<br>
⚫️ [CLS] Tokens_of_sentence_1 [SEP] Tokens_of_sentence_2 [SEP]<br>
> **Correct!** That's correct!

⚪️ [CLS] Tokens_of_sentence_1 [SEP] Tokens_of_sentence_2

**4. What are the benefits of the `Dataset.map()` method?**<br>
⚫️ The results of the function are cached, so it won't take any time if we re-execute the code.
> **Correct!** That is indeed one of the neat benefits of this method! It's not the only one, though...

⚫️ It can apply multiprocessing to go faster than applying the function on each element of the dataset.
> **Correct!** This is a neat feature of this method, but it's not the only one!

⚫️ It does not load the whole dataset into memory, saving the results as soon as one element is processed.
> **Correct!** That's one advantage of this method. There are others, though!

**5. What does dynamic padding mean?**<br>
⚪️ It's when you pad the inputs for each batch to the maximum length in the whole dataset.<br>
⚫️ It's when you pad your inputs when the batch is created, to the maximum length of the sentences inside that batch.
> **Correct!** That's correct! The "dynamic" part comes from the fact that the size of each batch is determined at the time of creation, and all your batches might have different shapes as a result.

⚪️ It's when you pad your inputs so that each sentence has the same number of tokens as the previous one in the dataset.

**6. What is the purpose of a collate function?**<br>
⚪️ It ensures all the sequences in the dataset have the same length.<br>
⚫️ It puts together all the samples in a batch.
> **Correct!** Correct! You can pass the collate function as an argument of a `DataLoader`. We used the `DataCollatorWithPadding` function, which pads all items in a batch so they have the same length.

⚪️ It preprocesses the whole dataset.<br>
⚪️ It truncates the sequences in the dataset.

**7. What happens when you instantiate one of the `AutoModelForXxx` classes with a pretrained language model (such as `bert-base-uncased`) that corresponds to a different task than the one for which it was trained?**<br>
⚪️ Nothing, but you get a warning.<br>
⚫️ The head of the pretrained model is discarded and a new head suitable for the task is inserted instead.
> **Correct!** Correct. For example, when we used `AutoModelForSequenceClassification` with `bert-base-uncased`, we got warnings when instantiating the model. The pretrained head is not used for the sequence classification task, so it's discarded and a new head is instantiated with random weights.

⚪️ The head of the pretrained model is discarded.<br>
⚪️ Nothing, since the model can still be fine-tuned for the different task.

**8. What's the purpose of `TrainingArguments`?**<br>
⚫️ It contains all the hyperparameters used for training and evaluation with the `Trainer`.
> **Correct!** Correct.

⚪️ It specifies the size of the model.<br>
⚪️ It just contains the hyperparameters used for evaluation.<br>
⚪️ It just contains the hyperparameters used for training.

**9. Why should you use the 🤗 Accelerate library?**<br>
⚪️ It provides access to faster models.<br>
⚪️ It provides a high-level API so I don't have to implement my own training loop.<br>
⚫️ It makes our training loops work on distributed strategies.
> **Correct!** Correct! With 🤗 Accelerate, your training loops will work for multiple GPUs and TPUs.

⚪️ It provides more optimization functions.